# This notebook studies the relationship between fingerprints (i.e., sequence of predictions) and sessions

## We want to know how many sessions per fingerprints and viceversa

### Import dataset

In [ ]:
import pandas as pd

In [ ]:
PATH = "../Inference"
cyberlab = pd.read_csv(f"{PATH}/corpus_with_predictions.csv")
cyberlab.head(2)

### How many unique sequences of predicted classes?

In [ ]:
counter_fingerprints_cyberlab = cyberlab.value_counts("Models_predictions").reset_index().rename({0:"associated_sessions"}, axis = 1)
print(f"Number of unique model's predictions: {counter_fingerprints_cyberlab.shape[0]}")

### Obtain ECDF

#### This is the number of sessions associated per fingerprint

In [ ]:
import numpy as np
cdf_cyberlab = counter_fingerprints_cyberlab.value_counts("associated_sessions").sort_index()
cdf_cyberlab = np.cumsum(cdf_cyberlab)/np.sum(cdf_cyberlab)
cdf_cyberlab = cdf_cyberlab.reset_index(name = "cdf")

In [ ]:
import matplotlib.pyplot as plt 

fig, (ax_1) = plt.subplots(1, figsize=(5,4))
fontsize = 20
ax_1.plot(cdf_cyberlab.associated_sessions, cdf_cyberlab.cdf, linewidth = 2, color = "royalblue", label = "Cyberlab")

ax_1.set_xlabel('|sessions| per fingerprint', fontsize = fontsize)
ax_1.set_xscale("log")
ax_1.set_ylabel('ECDF fingerprint', fontsize = fontsize)
ax_1.yaxis.set_tick_params(labelsize=fontsize)
ax_1.xaxis.set_tick_params(labelsize=fontsize, rotation = 30)
ax_1.set_xticks([1, 10, 100, 1000, 10000])
ax_1.set_yticks(np.arange(.1, 1.1, .2))
ax_1.legend(fontsize = fontsize - 2, loc = "lower left")
ax_1.grid()


plt.tight_layout()
plt.savefig("./Inference_results/sessions_per_fingerprint.pdf")

#### Now obtain inverse information: 
- on the x_axis I the session ID; 
- on the y_axis, the number of other sessions with the same prediction as the one I am considering

In [ ]:
cyberlab = cyberlab.merge(counter_fingerprints_cyberlab, on = "Models_predictions").sort_values(by = "associated_sessions")

In [ ]:
counter_sessions_per_fingerprints_cyberlab = cyberlab.groupby("associated_sessions")["full_session"].count().reset_index().rename({"associated_sessions": "|sessions in the same family|", "full_session":"occurrences"}, axis = 1)

In [ ]:
counter_sessions_per_fingerprints_cyberlab["cdf_occurrences"] = 1 - np.cumsum(counter_sessions_per_fingerprints_cyberlab["occurrences"])/np.sum(counter_sessions_per_fingerprints_cyberlab["occurrences"])

In [ ]:
fig, (ax_2) = plt.subplots(1, figsize=(5,4))

ax_2.plot(counter_sessions_per_fingerprints_cyberlab["|sessions in the same family|"], counter_sessions_per_fingerprints_cyberlab["cdf_occurrences"], linewidth = 2, color = "royalblue", label = "Cyberlab")

ax_2.set_xlabel('|sessions| per fingerprint', fontsize = fontsize)
ax_2.set_xscale("log")
ax_2.set_ylabel('% remaining sessions', fontsize = fontsize)
ax_2.yaxis.set_tick_params(labelsize=fontsize)
ax_2.xaxis.set_tick_params(labelsize=fontsize, rotation = 30)
ax_2.set_xticks([1, 10, 100, 1000, 10000])
ax_2.set_yticks(np.arange(.1, 1.1, .2))
ax_2.legend(fontsize = fontsize - 2, loc = "lower left")

ax_2.grid()
plt.tight_layout()
plt.savefig("./Inference_results/remaining_corpus.pdf")